# Saliva Processing

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from fau_colors import cmaps

from stresspose_analysis.datasets.mainstudy import MainStudyDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

In [ ]:
deploy_type = "local"

In [ ]:
config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
path_out = base_path.joinpath("saliva/processed")
bp.utils.file_handling.mkdirs([path_out])

In [ ]:
dataset = MainStudyDataset(base_path)
dataset

## Cortisol

### Samples

In [ ]:
cort_data = bp.io.saliva.load_saliva_plate(
    base_path.joinpath("saliva/raw/cortisol_values.xlsx"), saliva_type="cortisol", regex_str=r"(VP_\d+) (T\d)_(S\d)"
)

cort_data = cort_data.join(dataset.day_condition_map).reset_index()
cort_data = cort_data.drop(columns="day").set_index(["subject", "condition", "sample"])
cort_data.head()

In [ ]:
cort_data.to_csv(path_out.joinpath("cortisol_samples.csv"))

### Features

In [ ]:
cort_auc = bp.saliva.auc(cort_data, remove_s0=True, sample_times=dataset.sample_times_saliva, saliva_type="cortisol")

In [ ]:
cort_max_inc = bp.saliva.max_increase(cort_data, remove_s0=True, saliva_type="cortisol")
cort_max_inc_percent = bp.saliva.max_increase(cort_data, remove_s0=True, percent=True, saliva_type="cortisol")

In [ ]:
cort_slope = bp.saliva.slope(
    cort_data, sample_labels=["S1", "S4"], sample_times=dataset.sample_times_saliva, saliva_type="cortisol"
)

In [ ]:
cort_features = pd.concat([cort_auc, cort_max_inc, cort_max_inc_percent, cort_slope], axis=1)
cort_features = bp.saliva.utils.saliva_feature_wide_to_long(cort_features, saliva_type="cortisol")
cort_features.head()

In [ ]:
cort_features.to_csv(path_out.joinpath("cortisol_features.csv"))

## Amylase

### Samples

In [ ]:
amy_data = bp.io.saliva.load_saliva_plate(
    base_path.joinpath("saliva/raw/amylase_values.xlsx"), saliva_type="amylase", regex_str=r"(VP_\d+) (T\d)_(S\d)"
)

amy_data = amy_data.join(dataset.day_condition_map).reset_index()
amy_data = amy_data.drop(columns="day").set_index(["subject", "condition", "sample"])
amy_data.head()

In [ ]:
amy_data.to_csv(path_out.joinpath("amylase_samples.csv"))

### Features

In [ ]:
amy_auc = bp.saliva.auc(amy_data, remove_s0=True, sample_times=dataset.sample_times_saliva, saliva_type="amylase")

In [ ]:
amy_max_inc = bp.saliva.max_increase(amy_data, remove_s0=True, saliva_type="amylase")
amy_max_inc_percent = bp.saliva.max_increase(amy_data, remove_s0=True, percent=True, saliva_type="amylase")

In [ ]:
amy_slope = bp.saliva.slope(
    amy_data, sample_labels=["S1", "S4"], sample_times=dataset.sample_times_saliva, saliva_type="amylase"
)

In [ ]:
amy_features = pd.concat([amy_auc, amy_max_inc, amy_max_inc_percent, amy_slope], axis=1)
amy_features = bp.saliva.utils.saliva_feature_wide_to_long(amy_features, saliva_type="amylase")
amy_features.head()

In [ ]:
amy_features.to_csv(path_out.joinpath("amylase_features.csv"))

## Progesterone

In [ ]:
prog_estr_data = pd.read_excel(
    base_path.joinpath("saliva/raw/estradiol_progesterone_values.xlsx"), sheet_name=["progesterone", "estradiol"]
)

prog_estr_data = {key: value[["sample ID", key]].dropna() for key, value in prog_estr_data.items()}
prog_estr_data = {
    key: data.assign(**{"subject": prog_estr_data[key]["sample ID"].astype(int).apply(lambda s: f"VP_{s:02d}")})
    for key, data in prog_estr_data.items()
}
prog_estr_data = {key: data.drop(columns="sample ID").set_index("subject") for key, data in prog_estr_data.items()}

prog_estr_data = pd.concat({key: val[key] for key, val in prog_estr_data.items()}, axis=1)
prog_estr_data.head()

In [ ]:
prog_estr_data.to_csv(path_out.joinpath("progesterone_estradiol_samples.csv"))